In [1]:
! cd .. && python setup.py develop >/dev/null

/Volumes/Workspace/whylogs-python/notebooks


Constraints test whether values in the data are consistent with expectations.

For the moment, constraints apply only to numeric (fraction) features.  Constraints on string features may be applied to numeric statistics such as number_unique or maximum_length.


There are two types of constraints - ValueConstraint and SummaryConstraint. 
- ValueConstraints are applied as values are processed through whylogs.
- SummaryConstraints are applied to feature summaries.



In [8]:
from whylogs.core.statistics.constraints import SummaryConstraint, ValueConstraint, Op 
from whylogs.util.protobuf import message_to_json

v = ValueConstraint(Op.LT, 3.6)
print(message_to_json(v.to_protobuf()))

# constraints may have an optional name
s = SummaryConstraint('min', Op.LT, 300000, name='< 30K')
print(message_to_json(s.to_protobuf()))



{
  "name": "value LT 3.6",
  "value": 3.6,
  "op": "LT"
}
{
  "name": "< 30K",
  "field": "min",
  "value": 300000.0,
  "op": "LT"
}


Constraints are internallyconverted to python lambda functions that are faster to evaluate.

In [9]:
print(s.func)

<function <lambda>.<locals>.<lambda> at 0x7fd5871b7280>


Constraints may be applied across multiple features, and a single feature may have multiple constraints.

The `verbose` option will log every failure of a constraint.

In [10]:
from whylogs.core.statistics.constraints import DatasetConstraints

conforming_loan = ValueConstraint(Op.LT, 548250)
smallest_loan = ValueConstraint(Op.GT, 2500.0, verbose=True)

high_fico = ValueConstraint(Op.GT, 4000)

non_negative = SummaryConstraint('min', Op.GE, 0)

dc = DatasetConstraints(None, value_constraints={'loan_amnt':[conforming_loan, smallest_loan], 'fico_range_high':[high_fico]},
                              summary_constraints={'annual_inc':[non_negative]})
print(message_to_json(dc.to_protobuf()))


{
  "valueConstraints": {
    "loan_amnt": {
      "constraints": [
        {
          "name": "value LT 548250",
          "value": 548250.0,
          "op": "LT"
        },
        {
          "name": "value GT 2500.0",
          "value": 2500.0,
          "op": "GT"
        }
      ]
    },
    "fico_range_high": {
      "constraints": [
        {
          "name": "value GT 4000",
          "value": 4000.0,
          "op": "GT"
        }
      ]
    }
  },
  "summaryConstraints": {
    "annual_inc": {
      "constraints": [
        {
          "name": "summary min GE 0/None",
          "field": "min",
          "value": 0.0,
          "op": "GE"
        }
      ]
    }
  }
}


The assembled DatasetConstraints may be applied to a whylogs logging session.

In [14]:
import os.path
import pandas as pd
import numpy as np
from whylogs import get_or_create_session
from whylogs.logs import display_logging

# turn on logging to show verbose constraints.
display_logging('info')

session = get_or_create_session()
data_file = "lending_club_1000.csv"
data = pd.read_csv(os.path.join(data_file))
profile = session.log_dataframe(data, 'test.data', constraints=dc)


2021-02-10 16:05:09,072 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 2400.0
2021-02-10 16:05:09,074 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 2300.0
2021-02-10 16:05:09,075 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 2000.0
2021-02-10 16:05:09,077 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 2400.0
2021-02-10 16:05:09,078 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 1600.0
2021-02-10 16:05:09,079 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 1000.0
2021-02-10 16:05:09,081 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed on value 1500.0
2021-02-10 16:05:09,083 - whylogs.core.statistics.constraints - INFO - value constraint value GT 2500.0 failed 

`DatasetConstraints.report()` API can report failures as a list of tuples.

In [6]:
from tabulate import tabulate

def indent(txt, spaces=4):
    return "\n".join(" " * spaces + ln for ln in txt.splitlines())

def format_report(r):
    # report failures in tabular form
    print("Constraint failures by feature - ")
    for c,r in r:
        print(f"{c}:")
        print(indent(tabulate(r, tablefmt="plain", headers=['test_name', 'total_run', 'failed'])))
        
format_report(dc.report())


Constraint failures by feature - 
loan_amnt:
    test_name          total_run    failed
    value LT 548250         1000         2
    value GT 2500.0         1000        20
fico_range_high:
    test_name        total_run    failed
    value GT 4000         1000      1000
annual_inc:
    test_name                total_run    failed
    summary min GE 0/None            0         0


So far we have only seen the value constraints applied during WhyLogs processing.

What about the Summary constraint defined above?

The `apply_summary_constraints()` API will apply summary constraints to a existing profile.  Without an argument, this applies constraints that were supplied when the profile was created.  Use the `constraints` to pass in your own map of summary constraints that should be applied.

In [16]:
print("Apply existing constraints to summary:")
r = profile.apply_summary_constraints()
format_report(r)

print("\n\n")
print("Apply new constraints to summary:")
non_negative = SummaryConstraint('min', Op.LT, 0)
r = profile.apply_summary_constraints(summary_constraints={'funded_amnt':[non_negative]})
format_report(r)

Apply existing constraints to summary:
Constraint failures by feature - 
annual_inc:
    test                     total_run    failed
    summary min GE 0/None            3         0



Apply new constraints to summary:
Constraint failures by feature - 
funded_amnt:
    test                     total_run    failed
    summary min LT 0/None            1         1


Some simple summary constraints can be automatically generated from a whylogs profiles.
By definition, generated constraints are valid for the profile used to generate them.


In [8]:
auto_constraints = profile.generate_constraints() # returns DatasetConstraint
print(message_to_json(auto_constraints.to_protobuf()))

{
  "properties": {
    "schemaMajorVersion": 1,
    "schemaMinorVersion": 1,
    "sessionId": "1ba3efce-c03a-4ea3-a525-120dd3e472bc",
    "sessionTimestamp": "1612971191517",
    "tags": {
      "name": "test.data"
    },
    "dataTimestamp": "0",
    "metadata": {}
  },
  "summaryConstraints": {
    "annual_inc_joint": {
      "constraints": [
        {
          "name": "summary min GT 0/None",
          "field": "min",
          "value": 0.0,
          "op": "GT"
        }
      ]
    },
    "dti": {
      "constraints": [
        {
          "name": "summary min GT 0/None",
          "field": "min",
          "value": 0.0,
          "op": "GT"
        }
      ]
    },
    "policy_code": {
      "constraints": [
        {
          "name": "summary min GT 0/None",
          "field": "min",
          "value": 0.0,
          "op": "GT"
        }
      ]
    },
    "last_fico_range_high": {
      "constraints": [
        {
          "name": "summary min GT 0/None",
          "field": 